In [1]:
import os
import importlib
import torch

from voc12 import my_dataloader
from torch.utils.data import DataLoader
from tqdm import tqdm
from utility.image_util import *

In [2]:
cam_network = "net.resnet50_cam"
rgbd_cam_weights_name = "../sess/voc_sess/resnet50_rgbd_cam"

val_list = "voc12/val.txt"
voc12_root = "../Dataset/VOC2012/"
depth_root = "../result/depth_img/"

device = "cuda"

In [3]:
dataset = my_dataloader.VOC12_DepthClassificationDataset(val_list, voc12_root, depth_root)


In [4]:
data_loader = DataLoader(dataset)



In [6]:
def contains_nan(tensor):
    return torch.isnan(tensor).any().item()

for step, pack in tqdm(enumerate(data_loader)):
    img = pack['img']
    if contains_nan(img):
        print(step)
    

2913it [00:56, 51.29it/s]
